In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.datasets import load_iris

In [20]:
data = load_iris()

columns_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

iris_data = pd.DataFrame(np.hstack([data.data, data.target[:, np.newaxis]]), columns = columns_names)

У нас есть четыре признака, относящиеся к размерам цветка, и классы. На первом этапе мы разобьём выборку на две части: тренировочную и валидационную, а позже рассмотрим, как добавить тестовую. Воспользуемся методом train_test_split() из библиотеки sklearn:

In [21]:
train, valid = train_test_split(iris_data, test_size=0.15, shuffle=True)

print('Размер исходной выборки: {}\nРазмер тренировочной выборки: {}\nРазмер валидационной выборки: {}'
      .format(iris_data.shape, train.shape, valid.shape)
)

Размер исходной выборки: (150, 5)
Размер тренировочной выборки: (127, 5)
Размер валидационной выборки: (23, 5)


Теперь визуализируем наше разбиение с помощью специального метода:

In [22]:
def visualize_train_valid_counts(init_data, train, valid):
    x = np.array([0, 1, 2])
    width = 0.2

    plt.figure(figsize=(15, 8))
    ax = plt.subplot(111)

    classes = list(init_data['class'].value_counts().index)

    ax.bar(x - width, list(init_data['class'].value_counts()[classes]), width, color='r', label='Исходные данные')
    ax.bar(x, list(train['class'].value_counts()[classes]), width, color='g', label='Тренировочная выборка')
    ax.bar(x, list(valid['class'].value_counts()[classes]), width, bottom=list(train['class'].value_counts()[classes]), color='b', label='Валидационная выборка')

    ax.set_ylim([0, 70])
    plt.xticks(x - width / 2, classes, fontsize=20)
    plt.yticks(fontsize=15)
    plt.ylabel('Кол-во примеров', fontsize=20)
    plt.minorticks_on()
    plt.grid(which='major', color='r')
    plt.grid(which='minor', linestyle=':', color='k')
    plt.legend(fontsize=15)

У нас три класса цветков, в каждом из которых получилось разное количество примеров с выборками, в третьем примере значительно больше примеров на валидационной выборке. Получилось **неравномерное распределение классов** после разбиения, так как мы не учли доли классов.

Проблема решается с помощью **stratified разбиения** или стратифицированного разбиения. В библиотеке sklearn есть нужный нам класс `StratifiedShuffleSplit`:

In [23]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15)
train_indices, valid_indices = [split for split in sss.split(iris_data.iloc[:, :4], iris_data.iloc[:, 4])][0]
s_train = iris_data.iloc[train_indices]
s_valid = iris_data.iloc[valid_indices]

## Практика

### Задание 4.3.1
Загрузите данные [train.csv](https://drive.google.com/file/d/1wbB8sqdz667_SUDISF57qSaSAUhtwlmP/view), разделите выборку на обучающую и тестовую части в соотношении 70% на 30% без перемешивания. Найдите среднее значение для признака `payment_amount` и запишите его в переменную `result`. Обратите внимание, что среднее нужно считать по тестовой выборке.

In [24]:
from sklearn.model_selection import train_test_split
vis_data = pd.read_csv("./Unit_5_data/train.csv", encoding = 'ISO-8859-1', low_memory = False)
# Напишите ваш код ниже

train, test = train_test_split(vis_data, test_size=0.3,shuffle=False)
result = test.payment_amount.mean()

## Задание 4.4.5
Допустим, у вас есть выборки `y_true = [1.23, 2.35, 2.75]` и `y_pred = [1.01, 12.3, 2.74]`. Посчитайте метрику `RMSE`, округлите до сотых и запишите результат. Например, 3.14

In [25]:
y_true = np.array([1.23, 2.35, 2.75])
y_pred = np.array([1.01, 12.3, 2.74])

In [26]:
ans = np.sqrt(((y_true-y_pred)**2).sum()/3)
round(ans,2)

5.75

In [27]:
ans2 = np.sqrt(mean_squared_error(y_true, y_pred))
ans2
round(ans2,2)

5.75

## Задание 4.5.1
Загрузите данные `train.csv`, оставьте в данных только признаки `'fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due'`, затем избавьтесь от пропусков. Целевой переменной будет `'balance_due'`, разделите данные на обучающую и тестовую выборки в соотношении 70% / 30% без перемешивания. Обучите линейную регрессию из `scikit-learn` и запишите в переменную `result` значение метрики **RMSE** на тестовой выборке. **RMSE** означает Rooted Mean Squared Error. Rooted означает, что из значения метрики был взят корень.

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
# vis_data = pd.read_csv("./train.csv", encoding = 'ISO-8859-1', low_memory = False)
vis_data = pd.read_csv("./Unit_5_data/train.csv", encoding = 'ISO-8859-1', low_memory = False)

In [29]:
columns_to_study = ['fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due']
vis_data = vis_data[columns_to_study]
vis_data.dropna(inplace = True)
X,y  = vis_data.drop(columns='balance_due'), vis_data.balance_due
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7,shuffle=False)
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
result = np.sqrt(mean_squared_error(y_test,y_pred))

## Задание 4.6.2
Используя библиотеку `sklearn`, найдите метод для подсчёта метрики . Создайте два одинаковых списка, состоящих из одинаковых 100 чисел. Замените в одном списке одно число на любое другое, отправьте эти списки в метод для подсчёта $accuracy$ . Что выдаст алгоритм на выходе? Ответ запишите с точностью до сотых, в качестве разделителя используйте точку.

In [30]:
from sklearn.metrics import accuracy_score

y_true,y_pred = [1]*100,[1]*100
y_pred[99] = 0
ans = accuracy_score(y_true,y_pred)
round(ans,2)

0.99

## Задание 4.6.7
На этапе вычисления метрик вы получили результаты precision = 0.75 и recall = 0.6. Посчитайте значение метрики $f1$. Округлите до сотых и запишите результат.

In [31]:
precision = 0.75
recall = 0.6
ans = 2*precision*recall/(precision+recall)
round(ans,2)

0.67

## Задание 4.6.8
Допустим, у вас есть выборки `y_true = [0, 0, 1, 1, 1, 1, 0, 1]` и `y_pred = [0, 1, 0, 0, 1, 1, 0, 1]`. Посчитайте значение метрики $f1$. Округлите до сотых и запишите результат.

In [32]:
from sklearn.metrics import f1_score

y_true = np.array([0, 0, 1, 1, 1, 1, 0, 1])
y_pred = np.array([0, 1, 0, 0, 1, 1, 0, 1])
ans = f1_score(y_true,y_pred)
round(ans,2)

0.67

## Задание 4.6.9
Допустим, у вас есть выборки `y_true = [0, 0, 1, 1, 1, 1, 0, 1]` и `y_pred = [0, 1, 0, 0, 1, 1, 0, 1]`. Посчитайте значение метрики $precision$ . Округлите до сотых и запишите результат.

In [33]:
from sklearn.metrics import precision_score

y_true = np.array([0, 0, 1, 0, 0, 1, 0])
y_pred = np.array([1, 1, 1, 0, 1, 1, 0])
ans = precision_score(y_true,y_pred)
round(ans,2)

0.4

In [34]:
(y_true*y_pred).sum()/y_pred.sum()

0.4

## Задание 4.6.10
Допустим, у вас есть выборки `y_true = [0, 0, 1, 0, 0, 1, 0]` и `y_pred = [1, 1, 1, 0, 1, 1, 0]`. Посчитайте значение метрики $recall$. Округлите до сотых и запишите результат.

In [35]:
from sklearn.metrics import recall_score

y_true = np.array([0, 0, 1, 0, 0, 1, 0])
y_pred = np.array([1, 1, 1, 0, 1, 1, 0])
ans = recall_score(y_true,y_pred)
round(ans,2)

1.0

# Практика для метрик классификации

В предложенном датасете есть ряд свойств, определенных по аудиозаписям:

**meanfreq**: средняя частота голоса (в кГц)  
**sd**: стандартное отклонение частоты голоса  
**median**: медианная частота (в кГц)  
**Q25**: значение в первом квартиле (в кГц)  
**Q75**: значение в третьем квартиле (в кГц)  
**IQR**: интерквартильный размах (в кГц)  
**skew**: ассиметрия  
**kurt**: эксцесс  
**sp.ent**: спектральная энтропия  
**sfm**: энтропия Винера  
**mode**: мода частоты  
**centroid**: частотный центроид  
**meanfun**: средняя основная частота, измеренная по акустическому сигналу  
**minfun**:  минимальная основная частота, измеренная по акустическому сигналу  
**maxfun**: максимальная основная частота, измеренная в акустическом сигнале  
**meandom**: среднее значение доминирующей частоты, измеренной по акустическому сигналу  
**mindom**: минимум доминирующей частоты, измеренной в акустическом сигнале  
**maxdom**: максимум доминирующей частоты, измеренной в акустическом сигнале  
**dfrange**: диапазон доминантных частот, измеренное на звуковой сигнал  
**modindx**: индекс модуляции голоса  

## Задание
Попробуйте построить модель, предсказывающую пол обладателя записи голоса.

Для этого:

- Разделите выборку на обучающую и тренировочную с параметрами `test_size=0.3`, `random_state=42`.

- Нормализуйте признаки с помощью функции `StandardScaler()`. Учитывайте, что нормализация тестовой выборки производится по среднему и отклонению тренировочной, которую мы считаем репрезентативной относительно генеральной совокупности.

- Обучите модель логистической регрессии на подготовленных данных.

***Пояснение***
Тестовые данные не должны влиять на параметры нормализации. Нужно использовать `SCALER.TRANSFORM` вместо `SCALER.FIT_TRANSFORM`, чтобы применять параметры нормализации, рассчитанные для тренировочных данных. Иначе данные в трейне и в тесте будут нормализованы по - разному.

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('./Unit_5_data/voiceDataSet.csv')

X, y = data.drop(columns='label'),(data.label=='male').astype('int')

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=42)

scaler = StandardScaler()

X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test)

lr = LogisticRegression()
lr.fit(X_train_scl,y_train)

y_pred = lr.predict(X_test_scl)
y_pred_proba = lr.predict_proba(X_test_scl)

## Задание 4.7.1
Вычислите значение метрики $accuracy$ и введите полученное значение. Запишите ответ, округлив его до третьего знака после запятой.

In [37]:
ans = accuracy_score(y_test,y_pred)
round(ans,3)

0.973

## Мультиклассовая классификация 
Теперь познакомимся с новым алгоритмом классификации, а также снова потренируемся делить выборку и осуществлять кросс-валидацию. Кроме того, посмотрим, как можно оценить качество классификации для случая, когда наша классификация не бинарная, т.е. у нас несколько классов.

В этом кейсе мы попробуем определять типы стекла по его характеристикам.

Наша классификация мультиклассовая, поэтому мы должны посмотреть, сколько у нас классов. Если вы вообще ничего не знаете о задаче в плане количества классов, обязательно начинайте с этого, поскольку это повлияет на ход решения. Особенно важно понять, бинарная классификация или нет.

In [38]:
data = pd.read_csv('./Unit_5_data/glass.csv')

In [39]:
X,y = data.drop(columns='Type'),data.Type

## Задание 4.7.2
Сколько классов стекла представлено в этой задаче?

In [40]:
data.Type.nunique()

6

Теперь воспользуемся k-fold валидацией на пяти разбиениях и обучим модель:

In [41]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold
model=KNeighborsClassifier(n_neighbors=3)
kf = KFold(n_splits=5)
cross_val_score(model, X, y, cv=kf, scoring="accuracy")

array([0.44186047, 0.65116279, 0.3255814 , 0.34883721, 0.07142857])

## Задание 4.7.3
Измените количество разбиений на 10. Вычислите среднее значение метрики $accuracy$ по 10 разбиениям и введите ниже, округлите до сотых.

In [42]:
# from google.colab import files
# files.upload()

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold

model=KNeighborsClassifier(n_neighbors=3)
kf = KFold(n_splits=10)

metrics = cross_val_score(model, X, y, cv=kf, scoring="accuracy")
ans = metrics.mean()
round(ans,2)

0.53

В следующей задаче мы будем диагностировать болезни сердца по различным медицинским параметрам пациентов.

Избавьтесь от выбросов, чтобы они не ухудшили качество нашей модели.  
Для того, чтобы избавиться от выбросов, уберите все данные, значения признаков которых отстают от первого или третьего квартиля более, чем на 1.5 межквартильного размаха.

In [43]:
data = pd.read_csv('./Unit_5_data/heart_fin1.csv',delimiter=';')

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   restecg   303 non-null    int64  
 6   thalach   303 non-null    int64  
 7   exang     303 non-null    int64  
 8   oldpeak   303 non-null    float64
 9   target    303 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 23.8 KB


In [45]:
X,y = data.drop(columns='target'),data.target

first_quartiles = np.quantile(X,0.25,axis=0)
third_quartiles = np.quantile(X,0.75,axis=0)
interquart = third_quartiles - first_quartiles
l_bounds = first_quartiles - 1.5*interquart
r_bounds = third_quartiles + 1.5*interquart

row_mask = ((X<l_bounds) | (X>r_bounds)).any(axis=1)

rows_to_drop = X.index[row_mask]

X,y  = X.drop(index=rows_to_drop),y.drop(index=rows_to_drop)

## Задание 4.7.4
Сколько наблюдений осталось после удаления выбросов?

In [46]:
len(y)

284

## Задание 4.7.5-4.7.6
Теперь разбейте выборку на тестовую и обучающую с параметрами test_size=0.15, random_state=5.

Обучите модели логистической регрессии ( c параметром max_iter=1000) и KNN (с количеством соседей, равным 3) на этих данных. Вычислите метрики качества $ROC AUC$.

In [47]:
from sklearn.metrics import roc_auc_score

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15, random_state=5)

lr = LogisticRegression(max_iter=1000)
knn = KNeighborsClassifier(n_neighbors=3)

lr.fit(X_train,y_train)
knn.fit(X_train,y_train)

y_pred_lr = lr.predict_proba(X_test)
y_pred_knn = knn.predict_proba(X_test)

lr_auc = roc_auc_score(y_test,y_pred_lr[:,1])
knn_auc = roc_auc_score(y_test,y_pred_knn[:,1])

round(lr_auc,2), round(knn_auc,2)

(0.82, 0.65)

# Практика

## Задание 4.9.1
Загрузите данные `train.csv`, оставьте в данных только признаки `'fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due', 'compliance'`, затем избавьтесь от пропусков.

Целевой переменной будет `'compliance'`, разделите данные на обучающую и тестовую выборки в соотношении 70%/30% без перемешивания.

Обучите `DecisionTreeClassifier` из `scikit-learn` с параметром `random_state=23`. Посчитайте значения метрики $F1$  на тренировочной выборке и на тестовой выборке. Вычтите из значения метрики на тренировочной выборки значение метрики на тестовой выборке и запишите результат в переменную `result`.

In [48]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# vis_data = pd.read_csv("./train.csv", encoding = 'ISO-8859-1', low_memory = False)
vis_data = pd.read_csv('./Unit_5_data/train.csv')

C:\Users\Manych\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [50]:
column_to_study = ['fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due', 'compliance']
vis_data = vis_data[column_to_study]

vis_data = vis_data.dropna()
X,y = vis_data.drop(columns = 'compliance'),vis_data.compliance

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,shuffle=False)

dtc = DecisionTreeClassifier(random_state=23)
dtc.fit(X_train,y_train)

y_pred_train = dtc.predict(X_train)
y_pred = dtc.predict(X_test)

score_train = f1_score(y_train,y_pred_train)
score_test = f1_score(y_test,y_pred)

result = score_train - score_test
result

0.08055341409051786

## Задание 4.11.1
Загрузите данные `train.csv`, оставьте в данных только признаки `'fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due', 'compliance'`, затем избавьтесь от пропусков.

Обучим модель на несбалансированных данных. Целевой переменной будет `'compliance'`, разделите данные на обучающую и тестовую выборки в соотношении 70%/30% без перемешивания.

Обучите `DecisionTreeClassifier` из `scikit-learn` с параметром `random_state=23`. Посчитайте значения метрики  на тренировочной выборке и на тестовой выборке.

Затем сделайте эту же выборку сбалансированной с помощью **undersampling**.

Для этого посчитайте количество примеров (`n`) класса-меньшинства, затем из класса-большинства возьмите `n` первых примеров.

То есть, в терминологии `Python`, возьмите срез (`slice`) от начала и до `n`, где `n` — количество примеров класса, которого в выборке представлено меньше.

Соедините две части выборки (с уменьшенным классом-большинством и с изначальным классом-меньшинством), сделайте точно такое же разбиение, как в задаче выше, и обучите такую же модель.

Посчитайте значение метрики  на новой тестовой выборке с помощью новой модели. Вычтите из значения метрики на тестовой части сбалансированной выборки значение метрики на тестовой части несбалансированной выборки и запишите результат в переменную `result`.

In [81]:
vis_data = pd.read_csv('./Unit_5_data/train.csv')

C:\Users\Manych\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [88]:
column_to_study = ['fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due', 'compliance']
vis_data = vis_data[column_to_study]
vis_data = vis_data.dropna()
X,y = vis_data.drop(columns = 'compliance'),vis_data.compliance
# unbalanced
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,shuffle=False)
dtc = DecisionTreeClassifier(random_state=23)
dtc.fit(X_train,y_train)
y_pred_unbal = dtc.predict(X_test)
score_test_unbal = f1_score(y_test,y_pred_unbal)
# balanced
index_pos = y[(y == 1)].index
index_neg = y[(y == 0)].index
n = len(index_pos)
index_downsempl = index_neg.append(index_pos[:n])
X_downsempl = X.loc[index_downsempl, :]
y_downsempl = y[index_downsempl]
X_train, X_test, y_train, y_test = train_test_split(X_downsempl, y_downsempl, test_size=0.3, shuffle=False)
dtc = DecisionTreeClassifier(random_state=23)
dtc.fit(X_train,y_train)
y_pred_bal = dtc.predict(X_test)
score_test_bal = f1_score(y_test,y_pred_bal)

result = score_test_bal - score_test_unbal

In [89]:
result

-0.580226904376013

In [91]:
vis_data_small = vis_data[['fine_amount', 'state_fee', 'late_fee', 
                           'discount_amount', 'balance_due', 'compliance']].dropna()
X = vis_data_small[['fine_amount', 'state_fee', 'late_fee', 'discount_amount', 'balance_due']]
y = vis_data_small['compliance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
clf = DecisionTreeClassifier(random_state=23)
clf.fit(X_train, y_train)
f1_imbalanced = f1_score(y_test, clf.predict(X_test))
zeros = y[y == 0.0]
ones = y[y == 1.0]
down_index = list(zeros.index[:len(ones)]) + list(ones.index)
X_down = X.loc[down_index, :]
y_down = y[down_index]
X_train, X_test, y_train, y_test = train_test_split(X_down, y_down, test_size=0.3, shuffle=False)
clf = DecisionTreeClassifier(random_state=23)
clf.fit(X_train, y_train)
f1_balanced = f1_score(y_test, clf.predict(X_test))
result = f1_balanced - f1_imbalanced

result

0.18272749967696011